In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import KNNImputer
import re
import matplotlib.pyplot as plt

from imblearn.over_sampling import SMOTE
from sklearn.datasets import make_classification

%matplotlib inline
# ignore warnings
import warnings
warnings.filterwarnings("ignore")


In [3]:
# df = pd.read_json('./data/dataSet_Culture_06102023-POINT.json')
df = pd.read_csv('./felix_clening.csv')

In [ ]:
# Extract the year from 'Analysis Date'
df['year'] = df['Analysis Date'].apply(lambda x: x.split('-')[0])

df['polygon_x'] = df['polygon'].apply(lambda x: x['x'])
df['polygon_y'] = df['polygon'].apply(lambda x: x['y'])
df['month'] = df['Analysis Date'].apply(lambda x: x.split('-')[1])
df['day'] = df['Analysis Date'].apply(lambda x: x.split('-')[2])

df['vegetation'] = (df['indextype'] == 'NDVI') & (df['averagevalue'] >= 0.15)
df = df.drop(['polygon', 'soil_id'] , axis = 1)

# Modify the 'combined' column to include year
df['combined'] = df['polygon_x'].astype(str) + '_' + df['polygon_y'].astype(str) + '_' + df['year'].astype(str)

# Assign unique ID based on the grouped column
df['id'] = df.groupby('combined').ngroup() + 1

# Drop the combined column and other temporary columns
df = df.drop(columns=['combined', 'polygon_x', 'polygon_y', 'year'])

In [ ]:
df = df[df['indextype'] == 'NDVI']
df = df.drop(columns=['indextype', 'year contour', 'month', 'day', 'vegetation', 'type_culture_name'])
df = df.rename(columns={'culture_name': 'class'})
df = df.rename(columns={'averagevalue': 'red'})
df = df.rename(columns={'Analysis Date': 'date'})
df['date'] = pd.to_datetime(df['date'])

In [ ]:
# Step 1: Create a mapping dictionary
group_mapping = {
    'Ячмень': 'Cereals', 'Пшеница': 'Cereals', 'Кукуруза': 'Cereals', 'Яровой': 'Cereals',
    'Соя': 'Legumes', 'Фасоль': 'Legumes', 'Клевер': 'Legumes', 'Люцерна': 'Legumes',
    'Картофель': 'Root Crops', 'Свекла': 'Root Crops', 'Морковь': 'Root Crops', 'Лук-Морковка': 'Root Crops',
    'Рапс': 'Oilseeds', 'Подсолнечник': 'Oilseeds', 'Софлор': 'Oilseeds',
    'Лук': 'Vegetables', 'Томат': 'Vegetables', 'Помидор': 'Vegetables', 'Капуста': 'Vegetables',
    'Арбуз': 'Melons',
    # Add more mappings as needed
}

# Step 2: Function to map class to group
def map_class_to_group(class_name):
    # Handle cases where the class_name is not in the dictionary (e.g., 'Сил', 'Экпарцет')
    return group_mapping.get(class_name, 'Miscellaneous')

# Step 3: Apply the function to create a new column
df['group_class'] = df['class'].apply(map_class_to_group)

# Show the updated DataFrame
print(df)

In [ ]:
label_encoder = LabelEncoder() 
  
# Encode labels in column 'class'. 
df['class']= label_encoder.fit_transform(df['class']) 
df['group_class']= label_encoder.fit_transform(df['group_class']) 
df['district_name']= label_encoder.fit_transform(df['district_name']) 
df['soil_name']= label_encoder.fit_transform(df['soil_name']) 
df['class'].unique() 

In [ ]:
# Convert date to multiple columns (year, month, day)
df['year'] = pd.to_datetime(df['date']).dt.year
df['month'] = pd.to_datetime(df['date']).dt.month
df['day'] = pd.to_datetime(df['date']).dt.day
df.drop('date', axis=1, inplace=True)

In [ ]:
# Pivot table
pivot_df = df.pivot_table(index='id', columns='month', values='red', aggfunc='mean')

# Fill NaN values (assuming you want to fill with zeros, adjust if needed)
pivot_df = pivot_df.fillna(0)

# Rename columns as needed
pivot_df.columns = [f'red_{col}_month' for col in pivot_df.columns]

# Reset the index so 'id' becomes a column
pivot_df = pivot_df.reset_index()

# Assuming each 'id' has a unique 'class', get the 'class' value for each 'id' and add to the pivot dataframe
pivot_df['class'] = pivot_df['id'].map(df.drop_duplicates(subset='id').set_index('id')['group_class'])
pivot_df['elevation_contour'] = pivot_df['id'].map(df.drop_duplicates(subset='id').set_index('id')['elevation_contour'])
pivot_df['district_name'] = pivot_df['id'].map(df.drop_duplicates(subset='id').set_index('id')['district_name'])
pivot_df['soil_name'] = pivot_df['id'].map(df.drop_duplicates(subset='id').set_index('id')['soil_name'])

# Reordering columns
pivot_df = pivot_df[['red_4_month', 'red_5_month', 'red_6_month', 'red_8_month', 'red_9_month', 'id', 'elevation_contour', 'district_name', 'soil_name', 'class']]


In [11]:
# Splitting the data
X = df.drop(['class'], axis=1)  # Features excluding 'id' and 'class'
y = df['class']  # Target variable

In [ ]:
# imputer = KNNImputer(n_neighbors=5)
# X = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

In [ ]:
class_counts = y.value_counts()
single_sample_classes = class_counts[class_counts <= 2].index
filter_mask = ~y.isin(single_sample_classes)
X = X[filter_mask]
y = y[filter_mask]

In [ ]:
y.value_counts()

In [ ]:

y = pd.Series(y)

# Print class distribution before SMOTE
print('Class distribution before SMOTE:\n', y.value_counts())

# Apply SMOTE
smote = SMOTE(sampling_strategy='auto', k_neighbors=2)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Convert y_resampled to a pandas Series (for value_counts)
y_resampled = pd.Series(y_resampled)

# Print class distribution after SMOTE
print('Class distribution after SMOTE:\n', y_resampled.value_counts())

In [10]:
df = df[df['class'] != 6]

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)

In [16]:
# Defining the parameter grid for GradientBoostingClassifier
param_dist = {
    'n_estimators': [10, 50, 100, 200, 500],
    'learning_rate': [0.001, 0.01, 0.1, 0.5, 1],
    'max_depth': [3, 4, 5, 6, 7],
    'subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
    'max_features': ['sqrt', 'log2', None]
}

# Instantiating GradientBoostingClassifier
gbm = GradientBoostingClassifier()

# Setting up RandomizedSearchCV
random_search = RandomizedSearchCV(
    gbm, param_distributions=param_dist, n_iter=100, scoring='accuracy', 
    cv=5, verbose=1, n_jobs=-1
)

# Fitting the RandomizedSearchCV object to the data
random_search.fit(X_train, y_train)

# Getting the best parameters and the best score
print("Best Parameters:", random_search.best_params_)
print("Best Score:", random_search.best_score_)

# Evaluate on the test set
print("Test Set Score:", random_search.score(X_test, y_test))

Fitting 5 folds for each of 100 candidates, totalling 500 fits
Best Parameters: {'subsample': 0.7, 'n_estimators': 500, 'max_features': None, 'max_depth': 5, 'learning_rate': 0.1}
Best Score: 0.5178843226788432
Test Set Score: 0.4935897435897436


In [14]:
print("Train Set Score:", random_search.score(X_train, y_train))

Train Set Score: 1.0


In [17]:
from sklearn.metrics import confusion_matrix, classification_report

# Make predictions on the test set
y_pred = random_search.predict(X_test)

# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Display confusion matrix
print("Confusion Matrix:")
print(conf_matrix)

# Calculate classification report
class_report = classification_report(y_test, y_pred)

# Display classification report
print("Classification Report:")
print(class_report)


Confusion Matrix:
[[11  1  7  1  2]
 [ 2 13  5  7  2]
 [ 6  4 24  1  7]
 [ 0 13  6  9  0]
 [ 1  7  7  0 20]]
Classification Report:
              precision    recall  f1-score   support

           1       0.55      0.50      0.52        22
           2       0.34      0.45      0.39        29
           3       0.49      0.57      0.53        42
           4       0.50      0.32      0.39        28
           5       0.65      0.57      0.61        35

    accuracy                           0.49       156
   macro avg       0.51      0.48      0.49       156
weighted avg       0.51      0.49      0.49       156

